In [ ]:
!pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pydirectinput as pdi
import time
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [ ]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
shooting_flag = False
rcross_flag=False
lcross_flag=False
didReturn=True
previous_event = None
events = []
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            rshoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            relbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            rwrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            rhip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            lshoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            lelbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            lwrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            lhip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            # Calculate angle
            angle_lelbow = calculate_angle(lshoulder, lelbow, lwrist)
            angle_relbow = calculate_angle(rshoulder, relbow, rwrist)
            # Visualize angle
            cv2.putText(image, str(angle_lelbow), 
                           tuple(np.multiply(lelbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(angle_relbow), 
                           tuple(np.multiply(relbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            valid_wrist_landmarks = rwrist is not None and lwrist is not None
            # Check if both wrists are above shoulders and no shooting event is ongoing
            if valid_wrist_landmarks:
                if rwrist[1] < rshoulder[1] and lwrist[1] < lshoulder[1]:
                    if angle_relbow > 165 or angle_lelbow > 165:
                        current_event="Shot Finished"
                        didReturn=False
                    else:
                        if not shooting_flag and didReturn:
                            current_event="Shooting"
                            shooting_flag = True
                else:
                    shooting_flag = False
                    didReturn=True
                if rwrist[1] > rhip[1] and rwrist[0] > rhip[0] and not rcross_flag:
                    current_event="right cross"
                    rcross_flag=True
                else:
                    rcross_flag=False

                if lwrist[1] > lhip[1] and lwrist[0] > lhip[0] and not lcross_flag:
                    current_event="left cross"
                    lcross_flag=True
                else:
                    lcross_flag=False

                if abs(rshoulder[0]-lshoulder[0]) < .2 and abs(rhip[0]-lhip[0]) < .1 and not stepback_flag:
                    current_event="stepback"
                    stepback_flag=True
                else:
                    stepback_flag=False
                    
                if current_event != previous_event:
                    if current_event:
                        events.append(current_event)
                        print(events)
                        if current_event == "Shooting":
                            pdi.keyDown('1')  # the key press
                        elif current_event == "Shot Finished":
                            pdi.keyUp('1')  # Release the key
                        elif current_event == "right cross":
                            pdi.press('9')
                        elif current_event == "left cross":
                            pdi.press('0')
                        elif current_event == "stepback":
                            pdi.press('8')
                    previous_event = current_event
                 
                

        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        1
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            
            break

    cap.release()
    cv2.destroyAllWindows()

# 